In [ ]:
import sys, os
sys.path.insert(0, "C:\\Users\\xdzl45\\workspace\\molearn\\src") #uncomment/edit as appropriate for your system
import molearn
from molearn.autoencoder import Autoencoder
import torch
from copy import deepcopy
import biobox as bb
import numpy as np
from molearn.analysis import MolearnAnalysis, MolearnGUI

# input files
networkfile = f'data{os.sep}checkpoint_epoch2872.ckpt'
training_set_file = f'data{os.sep}MurD_closed_open_strided.pdb'
test_set_file = f'data{os.sep}MurD_closed_apo_strided.pdb'

print("Loading network")
checkpoint = torch.load(networkfile, map_location= torch.device('cpu'))
net = Autoencoder(**checkpoint['network_kwargs'])
net.load_state_dict(checkpoint['model_state_dict'])

print("Loading data")
MA = MolearnAnalysis()
MA.set_network(net)
MA.set_train_data(training_set_file)
MA.set_test_data(test_set_file)

print("Scanning landscape error")
landscape_err_latent, landscape_err_3d, xaxis, yaxis = MA.scan_error(samples=50)

print("Scanning landscape with custom function")
def f_distance(crd, M, idx1, idx2):
    M.coordinates = deepcopy(crd)
    M.set_current(0)
    return np.sqrt(np.sum(M.points[idx2]-M.points[idx1])**2)

_, idx1 = MA.mol.atomselect("*", 1, "CA", get_index=True) # find index of first atom of interest
_, idx2 = MA.mol.atomselect("*", 300, "CA", get_index=True)  # find index of second atom of interest
dist, xaxis, yaxis = MA.scan_custom(f_distance, [MA.mol, idx1, idx2], "dist", samples=50)

In [ ]:
MG = MolearnGUI(MA)

---